### Ground Truth Depth Visualization

In [1]:
import os
import json
import numpy as np
import pyexr
import open3d as o3d
import matplotlib.pyplot as plt

def load_exr_depth(path):
    try:
        exr = pyexr.read(path)
        if exr.ndim == 3:
            return exr[:, :, 0]
        return exr
    except Exception as e:
        print(f"Failed to load {path}: {e}")
        return None

def backproject_depth_map(depth_map, K, cam_to_world):
    h, w = depth_map.shape
    u, v = np.meshgrid(np.arange(w), np.arange(h))
    valid = (depth_map > 0) & (depth_map < 100) & np.isfinite(depth_map)
    if not np.any(valid):
        return np.empty((0, 3)), np.empty((0,))

    u_valid = u[valid]
    v_valid = v[valid]
    z = depth_map[valid]
    x = (u_valid - K[0, 2]) * z / K[0, 0]
    y = (v_valid - K[1, 2]) * z / K[1, 1]

    points_cam = np.vstack([x, y, z]).T
    points_cam_h = np.hstack([points_cam, np.ones((points_cam.shape[0], 1))])
    points_world = (cam_to_world @ points_cam_h.T).T[:, :3]
    return points_world, z

# Load camera parameters
with open("0/camera_params.json", "r") as f:
    camera_params = json.load(f)

cam = camera_params["Camera_Front"]
depth_path = os.path.join("0", cam["depth_path"])
K = np.array(cam["intrinsic"])
extrinsic = np.array(cam["extrinsic"])
cam_to_world = np.linalg.inv(extrinsic)

depth_map = load_exr_depth(depth_path)
if depth_map is not None:
    if depth_map.ndim == 3:
        depth_map = depth_map[:, :, 0]

    gt_points, depths = backproject_depth_map(depth_map, K, cam_to_world)

    if gt_points.shape[0] > 0:
        print(f"Camera_Front: {len(gt_points)} points")

        # Normalize and enhance small differences across the full range
        norm_depths = (depths - depths.min()) / (depths.max() - depths.min() + 1e-8)
        contrast_depth = np.log1p(norm_depths * 9) / np.log1p(9)
        enhanced_depth = 1.0 - contrast_depth  # Near = bright

        # Use turbo colormap, same direction
        colormap = plt.get_cmap("turbo")
        colors = colormap(enhanced_depth)[:, :3]

        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(gt_points)
        pcd.colors = o3d.utility.Vector3dVector(colors)
        o3d.io.write_point_cloud("DepthMap.ply", pcd)

        print("Saved contrast-enhanced ground truth point cloud to DepthMap.ply")
    else:
        print("No valid points from Camera_Front")
else:
    print("Failed to load depth map for Camera_Front")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Camera_Front: 773210 points
Saved contrast-enhanced ground truth point cloud to DepthMap.ply
